In [1]:
from typing import TypedDict, Annotated
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core import tools

In [4]:
from dotenv import load_dotenv
import os
load_dotenv
groq_api_key : str = os.environ.get("GROQ_API_KEY")

In [3]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

In [ ]:
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model="meta-llama/llama-guard-4-12b"
)